## Initialization  

In [1]:
%load_ext autoreload
%autoreload 2
import os 
import sys
sys.path.insert(0, './src')
import time
import argparse
import yaml
import types
import copy, pprint
from time import sleep
from datetime import datetime
import numpy  as np
import torch  
import wandb
import pandas as pd
from utils.notebook_modules import initialize, init_dataloaders, init_environment, init_wandb, \
                                   training_prep, disp_dataloader_info,disp_info_1, \
                                   warmup_phase, weight_policy_training, disp_gpu_info

from utils.util import (print_separator, print_heading, timestring, print_loss, load_from_pickle) #, print_underline, 
#                       print_dbg, get_command_line_args ) 

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=6, linewidth=132)
pd.options.display.width = 132
# torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=None)
# sys.path.insert(0, '/home/kbardool/kusanagi/AdaSparseChem/src')
# print(sys.path)
# disp_gpu_info() 
os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Training.ipynb"


## Create Environment

### Parse Input Args  - Read YAML config file

In [ ]:
# RESUME_MODEL_CKPT = 'model_train_ep_25_seed_0088'

## For RESTARTING
##
# input_args = " --config yamls/chembl_3task_train.yaml " \
#              " --resume " \
#              " --exp_id      330i85cg" \
#              " --exp_name    0308_1204" \
#              " --exp_desc    Train with dropout 0.5" \
#              " --seed_idx    0 "\
#              " --batch_size  128" \
#              " --lambda_sparsity  0.01"\
#              " --lambda_sharing   0.01" 
## get command line arguments

In [2]:
##  For Initiating 
##
input_args = " --config yamls/chembl_3task_train.yaml " \
             " --exp_desc    6 lyrs,dropout 0.5, weight 105 bch/ep policy 105 bch/ep " \
             " --hidden_size   50 50 50 50 50 50   " \
             " --tail_hidden_size   50" \
             " --seed_idx            0" \
             " --batch_size        128" \
             " --task_lr          0.001" \
             " --backbone_lr      0.001" \
             " --policy_lr        0.01" \
             " --lambda_sparsity  0.02" \
             " --lambda_sharing   0.01" 

#              " --hidden_size   100 100 100 100 100 100" \
#              " --tail_hidden_size  100 " \
                         

In [3]:
opt, ns = initialize(input_args, build_folders = True)


  command line parms : 
------------------------
 config...................  yamls/chembl_3task_train.yaml
 exp_id...................  None
 exp_name.................  None
 folder_sfx...............  None
 exp_desc.................  6 lyrs,dropout 0.5, weight 105 bch/ep policy 105 bch/ep
 hidden_sizes.............  [50, 50, 50, 50, 50, 50]
 tail_hidden_size.........  50
 seed_idx.................  0
 batch_size...............  128
 backbone_lr..............  0.001
 task_lr..................  0.001
 policy_lr................  0.01
 decay_lr_rate............  None
 decay_lr_freq............  None
 lambda_sparsity..........  0.02
 lambda_sharing...........  0.01
 gpu_ids..................  [0]
 resume...................  False
 cpu......................  False



##################################################
################### READ YAML ####################
##################################################


 log_dir              create folder:  ../experiments/AdaSparseChem/50x6_

### Setup Dataloader and Model  

In [4]:
dldrs = init_dataloaders(opt)

disp_dataloader_info(dldrs)

environ = init_environment(ns, opt, is_train = True, policy_learning = False, display_cfg = False)

# ********************************************************************
# **************** define optimizer and schedulers *******************
# ********************************************************************                                
environ.define_optimizer(policy_learning=False)
environ.define_scheduler(policy_learning=False)


##################################################
############### CREATE DATALOADERS ###############
##################################################

 trainset.y_class                                   :  [(13331, 5), (13331, 5), (13331, 5)] 
 trainset1.y_class                                  :  [(13331, 5), (13331, 5), (13331, 5)] 
 trainset2.y_class                                  :  [(13331, 5), (13331, 5), (13331, 5)] 
 valset.y_class                                     :  [(4137, 5), (4137, 5), (4137, 5)]  
 testset.y_class                                    :  [(920, 5), (920, 5), (920, 5)]  
                                 
 size of training set 0 (warm up)                   :  13331 
 size of training set 1 (network parms)             :  13331 
 size of training set 2 (policy weights)            :  13331 
 size of validation set                             :  4137 
 size of test set                                   :  920 
                               Total           

In [5]:
print(f" Current LR: {environ.optimizers['alphas'].param_groups[0]['lr'] }")
print(f" Current LR: {environ.optimizers['weights'].param_groups[0]['lr']}")
print(f" Current LR: {environ.optimizers['weights'].param_groups[1]['lr']}")

 Current LR: 0.01
 Current LR: 0.001
 Current LR: 0.001


In [6]:
# environ.optimizers['weights'].param_groups[0]

###  Weights and Biases Initialization 

In [7]:
init_wandb(ns, opt, environment = environ)

print(f" PROJECT NAME: {ns.wandb_run.project}\n"
      f" RUN ID      : {ns.wandb_run.id} \n"
      f" RUN NAME    : {ns.wandb_run.name}") 

wandb: Currently logged in as: kbardool (use `wandb login --relogin` to force relogin)


12ys3f1b 0325_1454 AdaSparseChem


 PROJECT NAME: AdaSparseChem
 RUN ID      : 12ys3f1b 
 RUN NAME    : 0325_1454
 PROJECT NAME: AdaSparseChem
 RUN ID      : 12ys3f1b 
 RUN NAME    : 0325_1454


In [8]:
# ns.wandb_run.finish()

### Initiate / Resume Training Prep

In [9]:
if opt['train']['resume']:
    RESUME_MODEL_CKPT = ""
    RESUME_METRICS_CKPT = ""    
    print(opt['train']['which_iter'])
    print(opt['paths']['checkpoint_dir'])
    print(RESUME_MODEL_CKPT)
    # opt['train']['resume'] = True
    # opt['train']['which_iter'] = 'warmup_ep_40_seed_0088'
    print_separator('Resume training')
    loaded_iter, loaded_epoch = environ.load_checkpoint(RESUME_MODEL_CKPT, path = opt['paths']['checkpoint_dir'], verbose = True)
    print(loaded_iter, loaded_epoch)    
#     current_iter = environ.load_checkpoint(opt['train']['which_iter'])
    environ.networks['mtl-net'].reset_logits()
    val_metrics = load_from_pickle(opt['paths']['checkpoint_dir'], RESUME_METRICS_CKPT)
    # training_prep(ns, opt, environ, dldrs, epoch = loaded_epoch, iter = loaded_iter )

else:
    print_separator('Initiate Training ')

##################################################
############### Initiate Training  ###############
##################################################


### Training Preparation

In [10]:
training_prep(ns, opt, environ, dldrs )

 cuda available [0]
 set print_freq to length of train loader: 105
 set eval_iters to length of val loader  : 33


In [11]:
disp_info_1(ns, opt, environ)


 Num_blocks                : 6                                

 batch size                : 128 
 batches/ Weight trn epoch : 105 
 batches/ Policy trn epoch : 105                                 

 Print Frequency           : -1 
 Config Val Frequency      : 500 
 Config Val Iterations     : -1 
 Val iterations            : 33 
 which_iter                : warmup 
 train_resume              : False                                 
 
 fix BN parms              : False 
 Task LR                   : 0.001 
 Backbone LR               : 0.001                                 

 Sharing  regularization   : 0.01 
 Sparsity regularization   : 0.02 
 Task     regularization   : 1                                 

 Current epoch             : 0  
 Warm-up epochs            : 75 
 Training epochs           : 125


In [12]:
print(environ.disp_for_excel())


    folder: 50x6_0325_1454_plr0.01_sp0.02_sh0.01_lr0.001
    layers: 6 [50, 50, 50, 50, 50, 50] 
    
    middle dropout         : 0.5
    last dropout           : 0.5
    diff_sparsity_weights  : False
    skip_layer             : 0
    is_curriculum          : False
    curriculum_speed       : 3
    
    task_lr                : 0.001
    backbone_lr            : 0.001
    decay_lr_rate          : 0.75
    decay_lr_freq          : 40
    
    policy_lr              : 0.01
    policy_decay_lr_rate   : 0.75
    policy_decay_lr_freq   : 50
    lambda_sparsity        : 0.02
    lambda_sharing         : 0.01
    lambda_tasks           : 1
    
    Gumbel init_temp       : 4
    Gumbel decay_temp      : 0.965
    Gumbel decay_temp_freq : 16
    Logit init_method      : random
    Logit init_neg_logits  : None
    Logit hard_sampling    : False
    Warm-up epochs         : 75
    training epochs        : 125
    Data split ratios      : [0.725, 0.225, 0.05]



## Warmup Training

In [13]:
# environ.display_trained_policy(ns.current_epoch,out=sys.stdout)
# ns.stop_epoch_warmup = 10
# ns.warmup_epochs = 10
print(ns.warmup_epochs, ns.current_epoch)
print_heading(f" Last Epoch: {ns.current_epoch}   # of warm-up epochs to do:  {ns.warmup_epochs} - Run epochs {ns.current_epoch+1} to {ns.current_epoch + ns.warmup_epochs}", verbose = True)

75 0
------------------------------------------------------------------------
 Last Epoch: 0   # of warm-up epochs to do:  75 - Run epochs 1 to 75
------------------------------------------------------------------------ 



In [14]:
# warmup_phase(ns,opt, environ, dldrs, epochs = 25)
warmup_phase(ns,opt, environ, dldrs)

------------------------------------------------------------------------
 Last Epoch: 0   # of warm-up epochs to do:  75 - Run epochs 1 to 75
------------------------------------------------------------------------ 

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
    1 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |   10.5782   4.1595e-02   2.2045e-04   10.6200 |   0.69343   0.52137   0.51910   0.52092 |   10.4007   4.1595e-02   2.2045e-04    10.4426 |   6.2 |
    2 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |   10.3769   4.1595e-02   2.2045e-04   10.4187 |   0.69258   0.54596   0.54898   0.54551 |   10.3883   4.1595e-02   2.2045e-04    10.4301 |   6.2 |
    3 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |   10.3866   4.1595e-02   2.2045e-04   10.4285 |   0.68992   0.56623   0.57273   0.56581 |   10.3482   4.1595e-02   2.2045e-04 

   39 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    7.3170   4.1595e-02   2.2045e-04    7.3588 |   0.59234   0.74613   0.74761   0.74599 |    8.8803   4.1595e-02   2.2045e-04     8.9221 |   8.5 |
   40 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    6.7006   4.1595e-02   2.2045e-04    6.7424 |   0.58873   0.74851   0.75013   0.74838 |    8.8317   4.1595e-02   2.2045e-04     8.8735 |   7.0 |
   41 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    7.1674   4.1595e-02   2.2045e-04    7.2092 |   0.58744   0.74976   0.75154   0.74964 |    8.7992   4.1595e-02   2.2045e-04     8.8411 |   9.1 |
   42 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    6.8035   4.1595e-02   2.2045e-04    6.8453 |   0.58645   0.75102   0.75287   0.75090 |    8.8012   4.1595e-02   2.2045e-04     8.8431 |   7.5 |
   43 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    6.6833   4.1595e-02   2.2045e-04    6.7251 |   0.58623   0.75268   0.75433   0.75256 |    8.7920   4.1595e-02   2.2045e-04     8.8338 |   7

In [15]:
# warmup_phase(ns,opt, environ, dldrs, epochs = 25)

In [16]:
# ns.wandb_run.finish()

In [17]:
# ns.wandb_run.finish()

In [18]:
# environ.losses

In [19]:
# environ.val_metrics

#### display parms

In [20]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")
print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #
print(' current lr: ', environ.optimizers['alphas'].param_groups[0]['lr'],)
print(' current lr: ', environ.optimizers['weights'].param_groups[0]['lr'])
print(' current lr: ', environ.optimizers['weights'].param_groups[1]['lr'])

 Backbone Learning Rate      : 0.001
 Tasks    Learning Rate      : 0.001
 Policy   Learning Rate      : 0.01

 Sparsity regularization     : 0.02
 Sharing  regularization     : 0.01 

 Tasks    regularization     : 1   
 Gumbel Temp                 : 4.0000         
 Gumbel Temp decay           : 16
 current lr:  0.01
 current lr:  0.001
 current lr:  0.001


In [21]:
# environ.opt['train']['policy_lr'] = 0.01
# opt['train']['policy_lr']         = 0.01
# environ.opt['train']['lambda_sparsity'] = 0.1
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# environ.opt['train']['decay_temp_freq'] = 2
# print(environ.optimizers['alphas'].param_groups)
# print(environ.optimizers['weights'].param_groups)
# print('initial lr: ', environ.optimizers['alphas'].param_groups[0]['initial_lr'] , 'current lr: ', environ.optimizers['alphas'].param_groups[0]['lr'],)
# print('current lr: ', environ.optimizers['weights'].param_groups[0]['lr'])
# print('current lr: ', environ.optimizers['weights'].param_groups[1]['lr'])

## Weight & Policy Training

### Weight/Policy Training Preparation

In [ ]:
# ns.flag_warmup = True

In [22]:
if ns.flag_warmup:
    print_heading( f"** {timestring()} \n"
                   f"** Training epoch: {ns.current_epoch} iter: {ns.current_iter}   flag: {ns.flag} \n"
                   f"** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)\n"
                   f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
                   f"** Take checkpoint and block gradient flow through Policy net", verbose=True)
#     environ.define_optimizer(policy_learning=True)
#     environ.define_scheduler(policy_learning=True)
    ns.flag_warmup = False
    ns.flag = 'update_weights'
    environ.fix_alpha()
    environ.free_weights(opt['fix_BN'])

------------------------------------------------------------------------------------------------------------------------
** 2022-03-25 15:13:08:312655 
** Training epoch: 75 iter: 7875   flag: update_weights 
** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)
** Switch from Warm Up training to Alternate training Weights & Policy 
** Take checkpoint and block gradient flow through Policy net
------------------------------------------------------------------------------------------------------------------------ 



In [23]:
# num_train_layers = None 
# environ.opt['is_curriculum'] = True
# environ.opt['curriculum_speed'] = 4
# ns.num_train_layers = None

In [24]:
print(ns.current_epoch,ns.training_epochs)
print(f"ns.current_epoch           : {ns.current_epoch}") 
print(f"ns.current_iters           : {ns.current_iter} \n")  
print(f"ns.training_epochs         : {ns.training_epochs}") 
# print(f"ns.stop_epoch_training     : {ns.stop_epoch_training}")
print(f"Batches in weight epoch    : {ns.stop_iter_w}")
print(f"Batches in policy epoch    : {ns.stop_iter_a}")
print(f"num_train_layers           : {ns.num_train_layers}")
print()

75 125
ns.current_epoch           : 75
ns.current_iters           : 7875 

ns.training_epochs         : 125
Batches in weight epoch    : 105
Batches in policy epoch    : 105
num_train_layers           : None



In [25]:
ns.training_epochs = 250
print_loss(environ.val_metrics, title = f"[e] Last ep:{ns.current_epoch}  it:{ns.current_iter}")
# environ.display_trained_policy(ns.current_epoch)
# environ.display_trained_logits(ns.current_epoch)

[e] Last ep:75  it:7875 -  Total Loss: 8.5527     
Task: 8.5109   Sparsity: 4.15950e-02    Sharing: 2.20447e-04 


In [26]:
# ns.current_epoch = 200
# ns.flag 

In [27]:
print_heading(f" Last Epoch Completed : {ns.current_epoch}       # of epochs to run:  {ns.training_epochs} -->  epochs {ns.current_epoch+1} to {ns.training_epochs + ns.current_epoch}"
              f"\n policy_learning rate : {environ.opt['train']['policy_lr']} "
              f"\n lambda_sparsity      : {environ.opt['train']['lambda_sparsity']}"
              f"\n lambda_sharing       : {environ.opt['train']['lambda_sharing']}"
              f"\n curriculum training  : {opt['is_curriculum']}     cirriculum speed: {opt['curriculum_speed']}     num_training_layers : {ns.num_train_layers}", 
              verbose = True)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 75       # of epochs to run:  250 -->  epochs 76 to 325
 policy_learning rate : 0.01 
 lambda_sparsity      : 0.02
 lambda_sharing       : 0.01
 curriculum training  : False     cirriculum speed: 3     num_training_layers : None
------------------------------------------------------------------------------------------------------------------------ 



### Weight/Policy Training

In [28]:
# weight_policy_training(ns, opt, environ, dldrs, epochs = 100)
weight_policy_training(ns, opt, environ, dldrs)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 75   # of epochs to run:  250 -->  epochs 76 to 325    
 policy_learning rate : 0.01      
 lambda_sparsity      : 0.02
 lambda_sharing       : 0.01 
 curriculum training  : False     cirriculum speed: 3     num_training_layers : None
------------------------------------------------------------------------------------------------------------------------ 

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
   76 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    4.9859   4.1595e-02   2.2045e-04    5.0277 |   0.64633   0.74361   0.74800   0.74347 |    9.6836   4.1595e-02   2.2045e-04     9.7254 |  11.5 |
Previous best_epoch:     0   best iter:     0,   best_value: 0.00000
New      best_epoch:    76

Previous best_epoch:    85   best iter:  9975,   best_value: 0.77186
New      best_epoch:    86   best iter: 10080,   best_value: 0.78103
   86 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    4.2160   3.2840e-02   5.2308e-04    4.2494 |   0.63077   0.76261   0.76377   0.76247 |    9.4710   3.2865e-02   4.6720e-04     9.5043 |   9.7 |
   87 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    4.3695   3.2865e-02   4.6720e-04    4.4028 |   0.59963   0.77493   0.77724   0.77480 |    8.9900   3.2865e-02   4.6720e-04     9.0234 |  10.1 |
   87 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    4.6279   3.0570e-02   3.3570e-04    4.6588 |   0.61407   0.77079   0.77138   0.77069 |    9.2039   3.0551e-02   3.3904e-04     9.2348 |   9.8 |
   88 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    5.0554   3.0551e-02   3.3904e-04    5.0863 |   0.60927   0.77290   0.77398   0.77279 |    9.1172   3.0551e-02   3.3904e-04     9.1481 |   9.7 |
   88 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    4

Previous best_epoch:    99   best iter: 12915,   best_value: 0.78701
New      best_epoch:   100   best iter: 13125,   best_value: 0.78742

[e] Policy training epoch:100  it:13125 -  Total Loss: 8.9491     
Task: 8.9217   Sparsity: 2.70670e-02    Sharing: 3.27597e-04 

 epch: 100   softmax      s        softmax       s        softmax       s
 -----  ----------------- -    ----------------- -    ----------------- - 
   1    0.2392    0.7608  0    0.1471    0.8529  0    0.3036    0.6964  0
   2    0.4690    0.5310  0    0.2959    0.7041  0    0.4746    0.5254  0
   3    0.5637    0.4363  1    0.4218    0.5782  0    0.5655    0.4345  1
   4    0.4572    0.5428  0    0.3825    0.6175  0    0.3477    0.6523  0
   5    0.2017    0.7983  0    0.2342    0.7658  0    0.2216    0.7784  0
   6    0.4137    0.5863  0    0.3384    0.6616  0    0.2424    0.7576  0


Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     

  112 |   1.00e-03   1.00e-03   1.00e-02  3.725e+00 |    3.4824   2.3238e-02   3.5201e-04    3.5060 |   0.60120   0.79297   0.79432   0.79288 |    9.0244   2.3238e-02   3.5201e-04     9.0480 |   9.7 |
Previous best_epoch:   110   best iter: 15120,   best_value: 0.79295
New      best_epoch:   112   best iter: 15540,   best_value: 0.79297
  112 |   1.00e-03   1.00e-03   1.00e-02  3.725e+00 |    3.7427   2.3146e-02   2.8960e-04    3.7661 |   0.61109   0.79112   0.79278   0.79100 |    9.1692   2.3203e-02   3.0908e-04     9.1927 |   9.9 |
  113 |   1.00e-03   1.00e-03   1.00e-02  3.725e+00 |    2.9822   2.3203e-02   3.0908e-04    3.0057 |   0.60355   0.79461   0.79569   0.79451 |    9.0594   2.3203e-02   3.0908e-04     9.0829 |   9.7 |
Previous best_epoch:   112   best iter: 15540,   best_value: 0.79297
New      best_epoch:   113   best iter: 15750,   best_value: 0.79461
  113 |   1.00e-03   1.00e-03   1.00e-02  3.725e+00 |    3.4840   2.3226e-02   1.6251e-04    3.5074 |   0.60253   0.79512

  124 |   7.50e-04   7.50e-04   1.00e-02  3.595e+00 |    2.9742   2.1400e-02   2.1432e-04    2.9958 |   0.60938   0.79771   0.79910   0.79761 |    9.1079   2.1440e-02   3.0042e-04     9.1296 |   9.9 |
  125 |   7.50e-04   7.50e-04   1.00e-02  3.595e+00 |    3.2408   2.1440e-02   3.0042e-04    3.2625 |   0.60317   0.80051   0.80202   0.80042 |    9.0563   2.1440e-02   3.0042e-04     9.0780 |   9.8 |
  125 |   7.50e-04   7.50e-04   1.00e-02  3.595e+00 |    2.9202   2.1208e-02   4.2453e-04    2.9419 |   0.59943   0.80223   0.80264   0.80214 |    8.9962   2.1240e-02   3.3437e-04     9.0178 |   9.9 |
Previous best_epoch:   124   best iter: 18060,   best_value: 0.80090
New      best_epoch:   125   best iter: 18375,   best_value: 0.80223

[e] Policy training epoch:125  it:18375 -  Total Loss: 9.0178     
Task: 8.9962   Sparsity: 2.12404e-02    Sharing: 3.34368e-04 

 epch: 125   softmax      s        softmax       s        softmax       s
 -----  ----------------- -    ----------------- -    

...

  268 |   3.16e-04   3.16e-04   4.22e-03  2.608e+00 |    1.4447   2.5633e-02   1.4790e-04    1.4704 |   0.69385   0.81841   0.81901   0.81832 |   10.3712   2.5633e-02   1.4790e-04    10.3970 |  10.1 |
  268 |   3.16e-04   3.16e-04   4.22e-03  2.608e+00 |    1.8079   2.4885e-02   1.0958e-04    1.8329 |   0.68839   0.81737   0.81806   0.81729 |   10.3546   2.4880e-02   1.1699e-04    10.3796 |  11.7 |
  269 |   3.16e-04   3.16e-04   4.22e-03  2.608e+00 |    1.7942   2.4880e-02   1.1699e-04    1.8192 |   0.68589   0.81922   0.81954   0.81913 |   10.3024   2.4880e-02   1.1699e-04    10.3274 |  10.1 |
  269 |   3.16e-04   3.16e-04   4.22e-03  2.608e+00 |    1.6609   2.5054e-02   1.4099e-04    1.6861 |   0.68719   0.81836   0.81916   0.81827 |   10.2986   2.5056e-02   1.3554e-04    10.3238 |   9.9 |
  270 |   3.16e-04   3.16e-04   4.22e-03  2.608e+00 |    1.1536   2.5056e-02   1.3554e-04    1.1788 |   0.68669   0.81927   0.81957   0.81918 |   10.2861   2.5056e-02   1.3554e-04    10.3113 |   9

  281 |   3.16e-04   3.16e-04   4.22e-03  2.608e+00 |    1.6160   2.5549e-02   1.3824e-04    1.6417 |   0.69113   0.81820   0.81852   0.81812 |   10.3530   2.5547e-02   1.2304e-04    10.3786 |  10.6 |
  282 |   3.16e-04   3.16e-04   4.22e-03  2.608e+00 |    1.5907   2.5547e-02   1.2304e-04    1.6164 |   0.69476   0.81782   0.81922   0.81772 |   10.3832   2.5547e-02   1.2304e-04    10.4088 |  11.5 |
  282 |   3.16e-04   3.16e-04   4.22e-03  2.608e+00 |    1.4821   2.5477e-02   1.8934e-04    1.5077 |   0.69681   0.81840   0.81860   0.81831 |   10.4439   2.5469e-02   1.9651e-04    10.4696 |   9.6 |
  283 |   3.16e-04   3.16e-04   4.22e-03  2.608e+00 |    1.6314   2.5469e-02   1.9651e-04    1.6570 |   0.69415   0.81824   0.81936   0.81815 |   10.4171   2.5469e-02   1.9651e-04    10.4428 |  10.4 |
  283 |   3.16e-04   3.16e-04   4.22e-03  2.608e+00 |    1.4545   2.5372e-02   1.7356e-04    1.4801 |   0.69677   0.81764   0.81886   0.81755 |   10.4520   2.5369e-02   1.7133e-04    10.4775 |   9

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
  296 |   3.16e-04   3.16e-04   4.22e-03  2.517e+00 |    1.6578   2.6338e-02   1.1780e-04    1.6843 |   0.69464   0.81961   0.82009   0.81950 |   10.4632   2.6338e-02   1.1780e-04    10.4896 |  10.0 |
  296 |   3.16e-04   3.16e-04   4.22e-03  2.517e+00 |    1.3425   2.5863e-02   1.3019e-04    1.3685 |   0.70317   0.81956   0.81973   0.81947 |   10.5229   2.5853e-02   1.2393e-04    10.5489 |  11.0 |
  297 |   3.16e-04   3.16e-04   4.22e-03  2.517e+00 |    2.0685   2.5853e-02   1.2393e-04    2.0944 |   0.70218   0.81892   0.81906   0.81884 |   10.5635   2.5853e-02   1.2393e-04    10.5895 |  10.7 |
  297 |   3.16e-04   3.16e-04   4.22e-03  2.517e+00 |    1.5901   2.5958e-02   1.2516e-04    1.6162 |   0.69340   0.81959   0.82000   0.81950 |   10.4199   2.5957e-02   1.1658e-04    10.4460 |   9

  310 |   2.37e-04   2.37e-04   4.22e-03  2.429e+00 |    1.4631   2.6993e-02   1.8714e-04    1.4903 |   0.70221   0.81987   0.81998   0.81979 |   10.5334   2.6993e-02   1.8714e-04    10.5606 |   9.3 |
  310 |   2.37e-04   2.37e-04   4.22e-03  2.429e+00 |    1.4034   2.7087e-02   1.0104e-04    1.4306 |   0.70065   0.81996   0.82016   0.81987 |   10.5246   2.7083e-02   1.6353e-04    10.5519 |   9.7 |

[e] Policy training epoch:310  it:57225 -  Total Loss: 10.5519     
Task: 10.5246   Sparsity: 2.70835e-02    Sharing: 1.63534e-04 

 epch: 310   softmax      s        softmax       s        softmax       s
 -----  ----------------- -    ----------------- -    ----------------- - 
   1    0.0843    0.9157  0    0.1037    0.8963  0    0.1568    0.8432  0
   2    0.5646    0.4354  1    0.3385    0.6615  0    0.4128    0.5872  0
   3    0.5774    0.4226  1    0.4071    0.5929  0    0.5386    0.4614  1
   4    0.4424    0.5576  0    0.4032    0.5968  0    0.3956    0.6044  0
   5    0.2249    0.

  322 |   2.37e-04   2.37e-04   3.16e-03  2.344e+00 |    1.5799   2.7602e-02   8.6529e-05    1.6076 |   0.71142   0.81909   0.81969   0.81900 |   10.6466   2.7608e-02   8.9878e-05    10.6743 |   9.3 |
  323 |   2.37e-04   2.37e-04   3.16e-03  2.344e+00 |    1.3504   2.7608e-02   8.9878e-05    1.3781 |   0.70003   0.81906   0.81919   0.81898 |   10.4638   2.7608e-02   8.9878e-05    10.4915 |   9.3 |
  323 |   2.37e-04   2.37e-04   3.16e-03  2.344e+00 |    1.6109   2.8326e-02   1.0537e-04    1.6393 |   0.71689   0.81959   0.82013   0.81950 |   10.7855   2.8331e-02   9.9723e-05    10.8139 |   9.2 |
  324 |   2.37e-04   2.37e-04   3.16e-03  2.344e+00 |    1.2638   2.8331e-02   9.9723e-05    1.2922 |   0.71093   0.81873   0.81908   0.81865 |   10.6603   2.8331e-02   9.9723e-05    10.6887 |   9.8 |
  324 |   2.37e-04   2.37e-04   3.16e-03  2.344e+00 |    1.4794   2.8049e-02   9.2703e-05    1.5075 |   0.71406   0.81962   0.81989   0.81954 |   10.6592   2.8040e-02   1.0270e-04    10.6874 |  10

In [ ]:
ns.best_epoch, ns.best_iter, ns.best_value

### Close WandB run

In [ ]:
wandb.finish()

In [ ]:
# ns.best_epoch = 0
# from utils.notebook_modules import wrapup_phase
# wrapup_phase(ns, opt, environ)

In [ ]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")


print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #
print(opt['train']['decay_temp_freq'])  


In [ ]:
# environ.opt['train']['policy_lr']       = 0.002
# environ.opt['train']['lambda_sparsity'] = 0.05
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# # environ.opt['train']['decay_temp_freq'] = 2
 

In [ ]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")


print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

print()
print( f" current_iters               : {ns.current_iter}")  
print( f" current_epochs              : {ns.current_epoch}") 
print( f" train_total_epochs          : {ns.training_epochs}") 
print( f" stop_epoch_training         : {ns.stop_epoch_training}")

## Post Training Stuff

In [ ]:
# pp.pprint(environ.losses)
# pp.pprint(environ.val_metrics)
environ.num_layers, environ.networks['mtl-net'].num_layers

In [ ]:
# pp.pprint(environ.val_metrics['total'])

In [ ]:
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch)
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# model_label   = 'model_train_ep_%d_seed_%04d' % (current_epoch, opt['random_seed'])
# metrics_label = 'metrics_train_ep_%d_seed_%04d.pickle' % (current_epoch, opt['random_seed'])
# environ.save_checkpoint(model_label, current_iter, current_epoch) 
# save_to_pickle(environ.val_metrics, environ.opt['paths']['checkpoint_dir'], metrics_label)
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# print_loss(current_iter, environ.losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, trn_losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, environ.val_metrics, title = f"[e] Policy training epoch:{current_epoch}    iter:")

In [ ]:
# environ.losses
# environ.val_metrics

In [ ]:
# environ.batch_data
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, 0, out=[sys.stdout])
# environ.display_parameters()

# with np.printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', precision=7, formatter={'float': lambda x: f"{x:12.5e}"}):
#     environ.print_logit_grads('gradients')

# environ_params = environ.get_task_specific_parameters()
# environ_params = environ.get_arch_parameters()
# environ_params = environ.get_backbone_parameters()
# print(environ_params)
# for param in environ_params:
#     print(param.grad.shape, '\n', param.grad)
#     print(param)

In [ ]:
environ.display_trained_logits(ns.current_epoch)
environ.display_trained_policy(ns.current_epoch)

In [ ]:
environ.display_test_sample_policy(ns.current_epoch, hard_sampling = True)
environ.display_train_sample_policy(ns.current_epoch, hard_sampling = True)

In [ ]:
# environ.define_optimizer(policy_learning=True)

In [ ]:
print(environ.optimizers['alphas'])
print(environ.optimizers['weights'])

In [ ]:
print('Policy  initial_lr : ', environ.optimizers['alphas'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['alphas'].param_groups[0]['lr'])
print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[0]['lr'])
print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[1]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[1]['lr'])

In [ ]:
wandb.run is None

In [ ]:
# opt['exp_instance'] = '0218_1358'     
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"
# print()
# opt['exp_instance'] = datetime.now().strftime("%m%d_%H%M")
# opt['exp_description'] = f"No Alternating Weight/Policy - training all done with both weights and policy"
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"

In [ ]:
# wandb.finish()

In [ ]:
# 
p = environ.get_current_state(0)

In [ ]:
pp.pprint(p)

### Post Warm-up Training stuff

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
environ.networks['mtl-net'].arch_parameters()

In [ ]:
p = environ.get_sample_policy(hard_sampling = False)
print(p)
p = environ.get_policy_prob()
print(p)
p = environ.get_policy_logits()
print(p)

# p = environ.get_current_policy()
# print(p)

In [ ]:
a = softmax([0.0, 1])
print(a)
sampled = np.random.choice((1, 0), p=a)
print(sampled)

In [ ]:
print(environ.optimizers['weights'])
print(environ.schedulers['weights'].get_last_lr())

In [ ]:
print('losses.keys      : ', environ.losses.keys())
print('losses[task]keys : ', environ.losses['task1'].keys())
pp.pprint(environ.losses)

In [ ]:
print( environ.val_metrics.keys())
# pp.pprint(val_metrics)
print(type(environ.val_metrics['aggregated']))
print()
print(type(environ.val_metrics['task1']['classification_agg']))
print()
pp.pprint(environ.val_metrics)

In [ ]:
# import pickle
# with open("val_metrics.pkl", mode= 'wb') as f:
#         pickle.dump(val_metrics, f)
    
# with open('val_metrics.pkl', 'rb') as f:    
#     tst_val_metrics = pickle.load(f)

In [ ]:
# print(environ.input.shape) 
# a = getattr(environ, 'task1_pred')
# yc_data = environ.batch['task1_data']
# print(yc_data.shape)
# yc_ind = environ.batch['task1_ind']
# print(yc_ind.shape)
# yc_hat_all = getattr(environ, 'task1_pred')
# print(yc_hat_all.shape)
# yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
# print(yc_hat_all.shape, yc_hat.shape)

# 
# environ.losses
# loss = {}
# for key in environ.losses.keys():
#     loss[key] = {}
#     for subkey, v in environ.losses[key].items():
#         print(f" key:  {key}   subkey: {subkey} ")
#         if isinstance(v, torch.Tensor):
#             loss[key][subkey] = v.data
#             print(f" Tensor  -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
#         else:
#             loss[key][subkey] = v
#             print(f" integer -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
# pp.pprint(tst_val_metrics)             

In [ ]:
# print('metrics.keys: ', environ.metrics.keys())
# print('metrics[task].keys: ', environ.metrics['task1'].keys())
# pp.pprint(environ.metrics['task1'])
# pp.pprint(environ.losses['task1']['total'])

In [ ]:
# title='Iteration'
# for t_id, _ in enumerate(environ.tasks):
#     task_key = f"task{t_id+1}"
# #     print_heading(f"{title}  {current_iter}  {task_key} : {val_metrics[task_key]['classification_agg']}", verbose = True)

#     for key, _  in val_metrics[task_key]['classification_agg'].items():
#         print('%s/%-20s'%(task_key, key), val_metrics[task_key]['classification_agg'][key], current_iter)
#         print(f"{task_key:s}/{key:20s}", val_metrics[task_key]['classification_agg'][key], current_iter)
#         print()
#             # print_current_errors(os.path.join(self.log_dir, 'loss.txt'), current_iter,key, loss[key], time.time() - start_time)

In [ ]:
# environ.print_loss(current_iter, start_time, metrics = val_metrics['loss'], verbose=True)
# print(opt['lambdas'])
# p = (opt['lambdas'][0] * environ.losses['tasks']['task1'])
# print(p)

# environ.print_val_metrics(current_iter, start_time, val_metrics , title='validation', verbose=True)    

In [ ]:
# print(current_iter)
# print_metrics_cr(current_iter, t1 - t0, None, val_metrics , True)
# environ.print_val_metrics(current_iter, start_time, val_metrics, title='validation', verbose = True)

In [ ]:
print(f" val_metric keys               : {val_metrics.keys()}")
print(f" loss keys                     : {val_metrics['loss'].keys()}")
print(f" task1 keys                    : {val_metrics['task1'].keys()}")
print(f" task1 classification keys     : {val_metrics['task1']['classification'].keys()}")
print(f" task1 classification_agg keys : {val_metrics['task1']['classification_agg'].keys()}")
print()
print(f" task1                       : {val_metrics['task1']['classification_agg']['loss']:5f}")
print(f" task2                       : {val_metrics['task2']['classification_agg']['loss']:5f}")
print(f" task3                       : {val_metrics['task3']['classification_agg']['loss']:5f}")
print(f" loss                        : {val_metrics['loss']['total']:5f}")
print(f" train_time                  : {val_metrics['train_time']:2f}")
print(f" epoch                       : {val_metrics['epoch']}")


### Post Weight + Policy Training Stuff 

In [ ]:
environ.networks['mtl-net'].backbone.layer_config

In [ ]:
num_blocks = 6
num_policy_layers = 6
gt =  torch.ones((num_blocks)).long()
gt0 =  torch.zeros((num_blocks)).long()
print(gt)
print(gt0)

loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
print(loss_weights)

In [ ]:
if environ.opt['diff_sparsity_weights'] and not environ.opt['is_sharing']:
    print(' cond 1')
    ## Assign higher weights to higher layers 
    loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
    print(f"{task_key} sparsity error:  {2 * (loss_weights[-num_blocks:] * environ.cross_entropy2(logits[-num_blocks:], gt)).mean()})")
    print_dbg(f" loss_weights :  {loss_weights}", verbose = True)
    print_dbg(f" cross_entropy:  {environ.cross_entropy2(logits[-num_blocks:], gt)}  ", verbose = True)
    print_dbg(f" loss[sparsity][{task_key}]: {self.losses['sparsity'][task_key] } ", verbose = True)

else:
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between \n Logits   : \n{logits[-num_blocks:]} \n and gt: \n{gt} \n", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-num_blocks:], gt)}")
    
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt[-1:])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt0[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt0[-1:])} \n")
    
    print('\n cond 3')    
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt[0:1])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt0[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt0[0:1])} \n")
        
        

In [ ]:
# flag = 'update_w'
# environ.fix_alpha
# environ.free_w(opt['fix_BN'])

flag = 'update_alpha'
environ.fix_weights()
environ.free_alpha()

In [ ]:
environ.networks['mtl-net'].num_layers

In [ ]:
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

train_total_epochs += 5

print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

In [ ]:
# print_metrics_cr(current_epoch, time.time() - t0, None, environ.val_metrics , num_prints)      

# num_prints += 1
# t0 = time.time()

# # Take check point
# environ.save_checkpoint('latest', current_iter)
# environ.train()
# #-------------------------------------------------------
# # END validation process
# #-------------------------------------------------------       
# flag = 'update_alpha'
# environ.fix_w()
# environ.free_alpha()

In [ ]:
# dilation = 2
# kernel_size = np.asarray((3, 3))
# upsampled_kernel_size = (kernel_size - 1) * (dilation - 1) + kernel_size
# print(upsampled_kernel_size)

In [ ]:
# environ.optimizers['weights'].param_groups[0]
# for param_group in optimizer.param_groups:
#     return param_group['lr']

In [ ]:
environ.schedulers['weights'].get_last_lr()

In [ ]:
current_state = {}
for k, v in environ.optimizers.items():
    print(f'state dict for {k} = {v}')
    current_state[k] = v.state_dict()
pp.pprint(current_state)

In [ ]:
current_state = {}
for k, v in environ.schedulers.items():
    print(f'state dict for {k} = {v}')
    print(v.state_dict())

### Losses and Metrics

In [ ]:
trn_losses = environ.losses

In [ ]:
print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# pp.pprint(environ.losses)
pp.pprint(trn_losses)

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
# environ.opt['train']['Lambda_sharing'] = 0.5
# opt['train']['Lambda_sharing'] = 0.5

# environ.opt['train']['policy_lr'] = 0.001
# opt['train']['policy_lr'] = 0.001

In [ ]:
environ.losses.keys()
pp.pprint(environ.losses)

In [ ]:
tmp = environ.get_loss_dict()
print(tmp.keys())
pp.pprint(tmp)

In [ ]:
print(opt['diff_sparsity_weights'])
print(opt['is_sharing'])
print(opt['diff_sparsity_weights'] and not opt['is_sharing'])
print(environ.opt['train']['Lambda_sharing'])
print(opt['train']['Lambda_sharing'])
print(environ.opt['train']['Lambda_sparsity'])
print(opt['train']['Lambda_sparsity'])
print(environ.opt['train']['policy_lr'])
print(opt['train']['policy_lr'])

### Policy / Logit stuff

In [ ]:
from scipy.special          import softmax

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=8,linewidth=132)

#### `get_task_logits(n)` Get logits for task group n

In [ ]:
task_logits = environ.get_task_logits(1)
print(task_logits)

#### `get_arch_parameters()`: Get last used logits from network

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

#### `get_policy_logits()`:  Get Policy Logits - returns same as `get_arch_parameters()`

In [ ]:
logs = environ.get_policy_logits()
for i in logs:
    print(i, '\n')
# probs = softmax(logs, axis= -1)
# for i in probs:
#     print(i, '\n')

#### `get_policy_prob()` : Gets the softmax of the logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
for i in policy_softmaxs:
    print(i, '\n')

#### `get_sample_policy( hard_sampling = False)` : Calls test_sample_policy of network with random choices based on softmax of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
policies,logits = environ.get_sample_policy(hard_sampling = False)

for l, p, s in zip(logits, policies, policy_softmaxs) :
    for  l_row, p_row, s_row in zip(l, p, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### `get_sample_policy( hard_sampling = True)` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
hard_policies, logits = environ.get_sample_policy(hard_sampling = True)

for p,l,s in zip(hard_policies, logits, policy_softmaxs) :
    for  p_row, l_row, s_row in zip(p, l, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### Print

In [ ]:
print(f" Layer    task 1      task 2      task 3")
print(f" -----    ------      ------      ------")
for idx, (l1, l2, l3) in enumerate(zip(hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1}       {l2}       {l3}")
    

    print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
def display_trained_policy(iter):

    policy_softmaxs = environ.get_policy_prob()
    policy_argmaxs = 1-np.argmax(policy_softmaxs, axis = -1)
    print(f"  Trained polcies at iteration: {iter} ")
    print(f"                   task 1                           task 2                         task 3        ")
    print(f" Layer       softmax        select          softmax        select          softmax        select   ")
    print(f" -----    ---------------   ------       ---------------   ------       ---------------   ------   ")
    for idx, (l1,l2,l3,  p1,p2,p3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2], policy_argmaxs[0], policy_argmaxs[1], policy_argmaxs[2]),1):
        print(f"   {idx}      {l1[0]:.4f}   {l1[1]:.4f}   {p1:4d}    {l2[0]:11.4f}   {l2[1]:.4f}   {p2:4d}    {l3[0]:11.4f}   {l3[1]:.4f}   {p3:4d}")

    print()
# print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
display_trained_policy(5)

In [ ]:
print(f"                        POLICIES (SOFTMAX)                                       task 3          ")
print(f" Layer    task1              task2            task3 softmax         softmax         argmax         softmax         argmax   ")
print(f" -----    -------------     -------------     -------------   ------   ")
for idx, (l1,l2,l3, h1,h2,h3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2],hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1[0]:.4f} {l1[1]:.4f}     {l2[0]:.4f} {l2[1]:.4f}     {l3[0]:.4f} {l3[1]:.4f}    {h3}")
    
print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
# print(policy_softmaxs[2], np.argmax(1-policy_softmaxs[2], axis = -1))
print(policy_softmaxs, np.argmax(policy_softmaxs, axis = -1))

#### `get_current_logits()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
logits  = (environ.get_current_logits())
for i in logits:
    print(i ,'\n')

#### `get_current_policy()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
pols  = (environ.get_current_policy())

for i in pols:
    print(i ,'\n')

#### `gumbel_softmax()`  

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', floatmode = 'maxprec_equal')
torch.set_printoptions(precision=8,linewidth=132)

In [ ]:
print(environ.temp)
# tau = environ.temp
tau = 1
for i in range(3): 
    logits_tensor = torch.tensor(logits[0])
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=tau, hard=False).cpu().numpy() 

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=tau, hard=True).cpu().numpy()
    
    for l, gs, gh in zip(lgts, gumbel_soft, gumbel_hard):
        print(f"   {l}   \t {gs}            \t {gh}")
#     print(lgts)
#     print(gumbel_soft)
#     print(gumbel_hard)
    print()

In [ ]:
for lgts in logits:
    logits_tensor = torch.tensor(lgts)
    print(lgts)
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=1, hard=False)
    print(gumbel_soft)

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=1, hard=True)
    print(gumbel_hard)
    print()

In [ ]:
smax = scipy.special.softmax(logs, axis =1)
# smax = np.array( 
# [[0.46973792, 0.530262  ],
#  [0.45025694, 0.549743  ],
#  [0.4443086 , 0.5556915 ],
#  [0.4138397 , 0.58616036],
#  [0.4140113 , 0.5859887 ],
#  [0.42114905, 0.57885087]])

print(smax.shape)
print(smax)
print(smax[0])
print(smax[0].sum())
print(np.random.choice((1,0), p =smax[0]))

In [ ]:
logs = np.array(
[[0.33064184, 0.42053092],
 [0.3532089 , 0.52056104],
 [0.3888512 , 0.5680909 ],
 [0.42039296, 0.694217  ],
 [0.4519742 , 0.73311865],
 [0.48401102, 0.7522658 ]],
)